In [45]:
import pandas as pd
import duckdb

zpravy = pd.read_pickle("pickle/zpravy.pkl")
vazby = pd.read_pickle("pickle/vazby.pkl")
dokumentace = pd.read_pickle("pickle/dokumentace.pkl")
materialy = pd.read_pickle("pickle/material.pkl")
vykony = pd.read_pickle("pickle/vykony.pkl")
vykpac = pd.read_pickle("pickle/vykpac.pkl")

res = duckdb.sql(
    """
    with zpravy_a_vazby as (
        select
            z.serial as zprava_serial,
            z.rc,
            z.ambnum,
            z.content,
            date_trunc('day', STRFTIME(STRPTIME(vazby.DATFR, '%d.%m.%Y %H:%M'), '%Y-%m-%dT%H:%M:%S')::timestamp) as datum_a_cas_zpravy,
            vazby.DEPARTM,
            vazby.WHO,
        from zpravy z
            left join vazby using (serial, ambnum)
    )
    , zpravy_a_vazby_a_dokumentace_tmp as (
        select distinct  -- why do we have duplicates
            datum_a_cas_zpravy,
            date_trunc('day', strptime(d.DATUM_CAS::text, '%Y%m%d_%H%M%S')) as datum_a_cas_dokumentace,
            zv.ambnum,
            zv.content,
            d.CISPAC,
        from zpravy_a_vazby zv
            asof left join dokumentace d
                on zv.ambnum = d.ambnum
                and date_trunc('day', strptime(d.DATUM_CAS::text, '%Y%m%d_%H%M%S')) >= datum_a_cas_zpravy
    )
    , zpravy_a_vazby_a_dokumentace as (
        select
            datum_a_cas_zpravy,
            ambnum,
            array_agg(distinct content) as contents,
            min(cispac) as cispac,
        from zpravy_a_vazby_a_dokumentace_tmp
        group by 
            datum_a_cas_zpravy,
            ambnum
    )
    , zpravy_a_vazby_a_dokumentace_a_vykony_tmp as (
        select
            date_trunc('day', strptime(datum::text, '%d.%m.%Y %H:%M')) as datum_vykonu,
            z.*,
            u.CDOKL,
            u.kod as kod_vykonu,
            u.odbornost,
            u.mnozstvi as mnozstvi_vykonu,
            u.body,
        from zpravy_a_vazby_a_dokumentace z
            left join vykony u on
                z.cispac = u.cispac
                and date_trunc('day', strptime(datum::text, '%d.%m.%Y %H:%M')) = datum_a_cas_zpravy
    )
    , zpravy_a_vazby_a_dokumentace_a_vykony as (
        select
            datum_a_cas_zpravy,
            contents,
            AMBNUM,
            cispac,
            array_agg(
                struct_pack(
                    CDOKL,
                    kod_vykonu,
                    odbornost,
                    mnozstvi_vykonu,
                    body
                )
            ) as vykony
        from zpravy_a_vazby_a_dokumentace_a_vykony_tmp
        group by 
            datum_a_cas_zpravy,
            contents,
            AMBNUM,
            cispac,
    )
    , zpravy_a_vazby_a_dokumentace_a_vykony_a_materialy_tmp as (
        select
            z.*,
            m.cdokl,
            m.kod as kod_materialu,
            m.mnozstvi as mnozstvi_materialu,
        from zpravy_a_vazby_a_dokumentace_a_vykony z
            left join materialy m on
                z.cispac = m.cispac
                and date_trunc('day', strptime(m.datum::text, '%d.%m.%Y %H:%M')) = datum_a_cas_zpravy
    )
    , zpravy_a_vazby_a_dokumentace_a_vykony_a_materialy as (
        select
            datum_a_cas_zpravy,
            contents,
            AMBNUM,
            cispac,
            vykony,
            array_agg(
                struct_pack(
                    cdokl,
                    kod_materialu,
                    mnozstvi_materialu
                )
            ) as materialy
        from zpravy_a_vazby_a_dokumentace_a_vykony_a_materialy_tmp
        group by 
            datum_a_cas_zpravy,
            contents,
            AMBNUM,
            cispac,
            vykony,
    )
    select *
    from zpravy_a_vazby_a_dokumentace_a_vykony_a_materialy
    """
).df()[["contents", "vykony", "materialy"]]
display(res)

,contents,vykony,materialy
0,[MUDr. Novák Tomáš\nC61 \nKontrola v amb. při ...,"[{'CDOKL': None, 'kod_vykonu': None, 'ODBORNOS...","[{'CDOKL': 444950, 'kod_materialu': '0222450',..."
1,[MUDr. Novák Tomáš\nC61 \nDnes apl. Firmagon 8...,"[{'CDOKL': 444954, 'kod_vykonu': 9561, 'ODBORN...","[{'CDOKL': 444954, 'kod_materialu': '0219085',..."
2,[MUDr. Hanušová Zuzana\nC19 \n16. cyklus Oxali...,"[{'CDOKL': 154984, 'kod_vykonu': 42022, 'ODBOR...","[{'CDOKL': 154984, 'kod_materialu': '0084090',..."
3,[MUDr. Danesh Adéla\nC184 \nPacient je schopen...,"[{'CDOKL': 166578, 'kod_vykonu': 42022, 'ODBOR...","[{'CDOKL': 166578, 'kod_materialu': '0000499',..."
4,[MUDr. Danesh Adéla\nC19 \nPacient je schopen ...,"[{'CDOKL': 172798, 'kod_vykonu': 42023, 'ODBOR...","[{'CDOKL': 172798, 'kod_materialu': '0250543',..."
...,...,...,...
66379,[MUDr. Bulová Alena\nC538 \nZ655 \nPřed aplika...,"[{'CDOKL': None, 'kod_vykonu': None, 'ODBORNOS...","[{'CDOKL': None, 'kod_materialu': None, 'mnozs..."
66380,[MUDr. Zeniou Alexis\nC678 \nKontrola u pac. n...,"[{'CDOKL': None, 'kod_vykonu': None, 'ODBORNOS...","[{'CDOKL': None, 'kod_materialu': None, 'mnozs..."
66381,[MUDr. Batko Stanislav\nC151 \nodběry 15.12.23...,"[{'CDOKL': None, 'kod_vykonu': None, 'ODBORNOS...","[{'CDOKL': None, 'kod_materialu': None, 'mnozs..."
66382,"[MUDr. Danesh Adéla\nC56 \n11.4.\nurea 7,7, kr...","[{'CDOKL': None, 'kod_vykonu': None, 'ODBORNOS...","[{'CDOKL': None, 'kod_materialu': None, 'mnozs..."


In [46]:
import math
import json

result = []
for i in range(len(res)):
    contents = res.iloc[i]["contents"]
    vykony = res.iloc[i]["vykony"]
    materialy = res.iloc[i]["materialy"]

    input = {"report": "\n".join(contents)}
    output = {"vykony": json.dumps(list(vykony)), "materialy": json.dumps(list(materialy))}
    result.append({"inputs": input, "outputs": output})

evalset = pd.DataFrame(result)
threadshold = math.floor(len(evalset) * 0.005)
training, test = evalset.iloc[:threadshold], evalset.iloc[threadshold:]

assert len(training) + len(test) == len(evalset)


In [47]:
from dotenv import load_dotenv
from langsmith import Client

# Load environment variables from a .env file
load_dotenv()

client = Client()

In [48]:
client.create_dataset(
    dataset_name="rakathon-oncoders",
)

LangSmithConflictError: Conflict for /datasets. HTTPError('409 Client Error: Conflict for url: https://api.smith.langchain.com/datasets', '{"detail":"Dataset with this name already exists."}')

In [43]:
training.to_dict(orient="records")

[{'inputs': {'report': 'MUDr. Čmejlová Vlastimila\nC504 \nsubj. stav idem, nemá zásadní potíže\nobj.  L prs zmenšen, v oblasti bradavky nález opět velikostní progrese nádoru cca na 3-4cm, axila s drobnými LU, WHO 1\n\nStaging: Markery 10/22 negativnmí\n\nMAMO 06.10.2022-08:54: MMG + UZ prsou\nPři srovnání s minulou dokumentací zjištěn bilat. stacionární mmg nález, vpravo reziduální ml. žlázy, vlevo tumorosní infiltrace retroareolárně se zesílením kůže, bilat. bez čerstvých ložiskových změn, shluků mikrokalcifikací či jiných známek malignity. \nAxilly bez zvětšených lymfatických uzlin.\nZávěr: stacionární nález rezidua tumoru vlevo retroareolárně, vpravo bez známek malignity.\n-----------------------------------------------------------------\nSONO 06.10.2022-07:14: UZ břicha St.p. CHCE. St.p. APPE. \nJátra nezvětšená,hladkých kontur, difuzně lehce vyšší echogenity parenchymu, v pravém jaterním laloku u confluens patrné hypoechogenní ložisko dnes vel. cca 13x11mm, další ložiska spolehliv

In [32]:
from dotenv import load_dotenv
from langsmith import Client

client.create_examples(
    dataset_name="rakathon-oncoders",
    examples=training.to_dict(orient="records"),
)


{'example_ids': ['c65f8dea-ce5a-4c22-9d9b-27a59ff6f4bf',
  '97d2cd37-0df6-4203-8415-5fe560b66739',
  'b50f151b-d74f-46b7-a850-e92c00fa2bee',
  '0761f866-4442-49ab-89f1-fc9a692874a3',
  '4e353c9c-bbf3-4db1-89f9-446de5026234',
  '6cfb14fa-11a6-449d-933a-45ed67b59d06',
  '82063a20-ea2c-4e1d-a3e2-5b08e4abdbad',
  '9967873f-54b4-4a43-9423-ae839a0d00e4',
  '9303a2a7-d087-4fce-b2d3-339ad0b4378b',
  'c9e3e5dd-1b63-47fa-8891-0da1bfe23eee',
  '45727487-bbbc-472c-a73d-fad68f37ace4',
  'db30aff7-bfe4-4b02-a411-115756d840b9',
  '7ca39202-7ec3-4ce1-b9e7-55644d3ba2d1',
  'c27c2191-c29d-46f1-a77c-c81c3cb05d92',
  '2df2fd39-189e-4757-b5cd-5cdd731ad851',
  '0d91ab04-9300-4bf1-956b-3fc626dc03e2',
  '4db2a24d-597f-4181-955e-0399b1915280',
  '91cd1553-c40c-43d2-b3fe-73d01829a7a1',
  '3f0b0306-b04e-4d60-b12a-bb5f42db293c',
  '0a4099d9-64d1-4d9e-8ecf-5af84ea1119f',
  '50203dc9-26ee-40ec-a553-3fe06da141b6',
  'ed038b2a-598a-4263-86d9-dc4c513d38f6',
  '671b57dd-72e4-4da1-8752-fda42c7e199f',
  '571575da-8ed2-47